In [1]:
import nest_asyncio
from dotenv import load_dotenv
from IPython.display import Markdown, display

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, ServiceContext, SimpleDirectoryReader

/home/justusbersten/bersten-rag/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# allows nested access to the event loop
nest_asyncio.apply()

In [3]:
# add your documents in this directory, you can drag & drop
input_dir_path = "documents/"

In [11]:

# setup llm & embedding model
llm=Ollama(model="llama3", request_timeout=300.0)
# embed_model = HuggingFaceEmbedding( model_name="Snowflake/snowflake-arctic-embed-m", trust_remote_code=True)
embed_model = HuggingFaceEmbedding( model_name="BAAI/bge-large-en-v1.5", trust_remote_code=True)

In [12]:
response = llm.complete("What is the capital of Germany?")
print(response)

The capital of Germany is Berlin.


In [13]:
# load data
loader = SimpleDirectoryReader(
            input_dir = input_dir_path,
            required_exts=[".pdf"],
            recursive=True
        )
docs = loader.load_data()

# Creating an index over loaded data
Settings.embed_model = embed_model
index = VectorStoreIndex.from_documents(docs, show_progress=True)

# Create the query engine, where we use a cohere reranker on the fetched nodes
Settings.llm = llm
query_engine = index.as_query_engine()

# ====== Customise prompt template ======
qa_prompt_tmpl_str = (
"Context information is below.\n"
"---------------------\n"
"{context_str}\n"
"---------------------\n"
"Given the context information above I want you to think step by step to answer the query in a crisp manner, incase case you don't know the answer say 'I don't know!'.\n"
"Query: {query_str}\n"
"Answer: "
)
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

# Generate the response
response = query_engine.query("What are some risk factors to be aware of?",)

Generating embeddings: 100%|██████████| 526/526 [00:47<00:00, 11.00it/s]


In [16]:
display(Markdown(str(response)))

Based on the provided context information, it appears that we are dealing with pricing and risk management of credit default swaps (CDS). Given this context, I would identify the following risk factors to be aware of:

1. **Survival probability**: The sensitivity of the premium leg to changes in survival probabilities is an important risk factor to consider.
2. **Zero hazard rates at nodes**: Changes in zero hazard rates at nodes can also impact the premium leg's value.
3. **Premiums only**: The sensitivity of premiums-only to changes in survival probabilities and zero hazard rates at nodes is another key risk factor.

These risk factors are highlighted in equations (91) and (94), which describe the sensitivity of the premium leg to these variables.

Please let me know if you'd like me to elaborate on any of these points or if there's anything else I can help with!